In [1]:
from bids import BIDSLayout

In [2]:
layout = BIDSLayout('/data/risk_precision/garcia_ruff_risk_precision_fmri/', validate=False)

In [3]:
df = layout.to_df()

In [14]:
df.subject.unique()

array([nan, '11927', '19131', '20466', '21246', '24613', '24626', '24933',
       '25427', '26753', '26867', '26905', '28051', '30130', '30451',
       '30733', '31422', '31724', '31814', '32091', '32829', '33156',
       '33464', '33524', '34501', '34626', '35091', '35092', '35114',
       '35177', '35202', '35721', '35982', '36874', '37073', '37257',
       '37268', '37305', '37368', '37982', '38312', '38422', '38629',
       '38726', '38940', '38972', '39010', '39051', '39089', '39115',
       '39152', '39166', '39180', '39203', '39283', '39509', '39572',
       '39575', '39627', '39693', '39903', '39960', '39999', '40048',
       '40359', '40425'], dtype=object)

In [126]:
import re
import json
metadata = df[df.suffix == 'sessions']

reg = re.compile('SNS_RPREC_(?P<subject_id>[X0-9]+)_[0-9]+')

def get_original_subject_id(row):
    with open(row.path) as f:
        d = json.load(f)[0]
        return reg.match(d['MRI_KEY']).group(1)
    
ori_ids = metadata.apply(get_original_subject_id, 1)
ori_ids[ori_ids == '21X'] = 21
ori_ids = ori_ids.astype(int).to_frame('id')
ori_ids = ori_ids.set_index(metadata.subject)

In [162]:
import numpy as np

In [180]:
fieldmaps = df[np.in1d(df['suffix'], ['magnitude', 'phase'])]

In [181]:
reg = re.compile('.*/sub-.+_ses-.+_acq-.+_rec-.+_(?P<echo>shortecho|longecho)_.+.nii')

In [182]:
fieldmaps['echo'] = fieldmaps.path.apply(lambda x: reg.match(x).group(1))

/Users/gdehol/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [183]:
fieldmaps['acquisition'] = fieldmaps['acquisition'].astype(int)

/Users/gdehol/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [184]:
fieldmaps = fieldmaps.set_index(['subject', 'acquisition', 'echo', 'suffix']).sort_index()

In [202]:
from nipype.pipeline import engine as pe

from nipype.interfaces import fsl
import nipype.interfaces.utility as niu
from niworkflows.interfaces.bids import DerivativesDataSink

In [206]:
def make_unwrap_wf(subject,
                   acq,
                  echo1_mag,
                  echo1_phase,
                  echo2_mag,
                  echo2_phase):
    
    
    wf = pe.Workflow(name=f'unwrap_{subject}_{acq}',
                    base_dir='/Users/gdehol/workflow_folders')
    
    inputnode = pe.Node(niu.IdentityInterface(fields=['echo1_mag',
                                                     'echo1_phase',
                                                      'echo2_mag',
                                                     'echo2_phase']),
                       name='inputnode')
    
    inputnode.inputs.echo1_mag = echo1_mag
    inputnode.inputs.echo1_phase = echo1_phase
    inputnode.inputs.echo2_mag = echo2_mag
    inputnode.inputs.echo2_phase = echo2_phase
    
    norm_phase1 = pe.Node(fsl.ImageMaths(op_string='-div 1000'), name='norm_phase1')
    wf.connect(inputnode, 'echo1_phase', norm_phase1, 'in_file')
    
    norm_phase2 = pe.Node(fsl.ImageMaths(op_string='-div 1000'), name='norm_phase2')
    wf.connect(inputnode, 'echo2_phase', norm_phase2, 'in_file')    
    
    
    unwrap_echo1 = pe.Node(fsl.PRELUDE(), name='unwrap_echo1')    
    unwrap_echo2 = pe.Node(fsl.PRELUDE(), name='unwrap_echo2')
    
    
    wf.connect(inputnode, 'echo1_mag', unwrap_echo1, 'magnitude_file')
    wf.connect(inputnode, 'echo2_mag', unwrap_echo2, 'magnitude_file')    
    
    wf.connect(norm_phase1, 'out_file', unwrap_echo1, 'phase_file')
    wf.connect(norm_phase2, 'out_file', unwrap_echo2, 'phase_file')        
    
    get_diff = pe.Node(fsl.ImageMaths(op_string = '-sub '), name='get_diff')
    
    wf.connect(unwrap_echo1, 'unwrapped_phase_file', get_diff, 'in_file')
    wf.connect(unwrap_echo2, 'unwrapped_phase_file', get_diff, 'in_file2')
    
    fake_fn = f'/tmp/sub-{subject}/fmap/sub-{subject}_acq-{acq}_run-{run}_phasediff.nii.gz'
    
    ds = pe.Node(DerivativesDataSink(suffix='phasediff', compress=True, out_path_base='sourcedata'), name='datasink')    
    ds.inputs.base_directory = f'/data/risk_precision/ds-risk/'
    ds.inputs.compress = True
    ds.inputs.source_file = fake_fn   
    
    wf.connect(get_diff, 'out_file', ds, 'in_file')
    
    ds_mag1 = pe.Node(DerivativesDataSink(suffix='magnitude1', 
                                          compress=True,
                                          base_directory='/data/risk_precision/ds-risk/',
                                          out_path_base='sourcedata'), name='ds_mag1')            
    ds_mag1.inputs.source_file = fake_fn   
    wf.connect(inputnode, 'echo1_mag', ds_mag1, 'in_file')
    
    
    ds_mag2 = pe.Node(DerivativesDataSink(suffix='magnitude2', 
                                          compress=True,
                                          base_directory='/data/risk_precision/ds-risk/',
                                          out_path_base='sourcedata'), name='ds_mag2')            
    ds_mag2.inputs.source_file = fake_fn   
    wf.connect(inputnode, 'echo1_mag', ds_mag2, 'in_file')    

    return wf

In [207]:
for (subject, session), d in fieldmaps.groupby(['subject', 'session']):
    
    run = 1
    
    for acq, d2 in d.groupby(['acquisition']):
        assert(len(d2) == 4)
        
        echo1_mag = d2.loc[(subject, acq, 'shortecho', 'magnitude'), 'path']
        echo1_phase = d2.loc[(subject, acq, 'shortecho', 'phase'), 'path']
        
        echo2_mag = d2.loc[(subject, acq, 'longecho', 'magnitude'), 'path']
        echo2_phase = d2.loc[(subject, acq, 'longecho', 'phase'), 'path']     
        
        ori_id = ori_ids.loc[subject].id
        
        print(subject, ori_id)
        
        wf = make_unwrap_wf(ori_id, acq, echo1_mag, echo1_phase, echo2_mag, echo2_phase)
        wf.run(plugin='MultiProc', plugin_args={'n_procs':6})
        
        run+=1

11927 16
190716-16:24:51,572 nipype.workflow INFO:
	 Workflow unwrap_16_3 settings: ['check', 'execution', 'logging', 'monitoring']
190716-16:24:51,580 nipype.workflow INFO:
	 Running in parallel.
190716-16:24:51,583 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 6/6.
190716-16:24:51,658 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_3.ds_mag2".


/Users/gdehol/miniconda3/lib/python3.7/site-packages/nipype/interfaces/fsl/preprocess.py:1887: UserWarning: This has not been fully tested. Please report any failures.
  warn('This has not been fully tested. Please report any failures.')


190716-16:24:51,694 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_3.ds_mag1".
190716-16:24:51,699 nipype.workflow INFO:
	 [Job 2] Cached (unwrap_16_3.norm_phase2).
190716-16:24:51,696 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_16_3.ds_mag2" in "/Users/gdehol/workflow_folders/unwrap_16_3/ds_mag2".
190716-16:24:51,704 nipype.workflow INFO:
	 [Job 4] Cached (unwrap_16_3.norm_phase1).
190716-16:24:51,703 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_3.ds_mag2".
190716-16:24:51,702 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_16_3.ds_mag1" in "/Users/gdehol/workflow_folders/unwrap_16_3/ds_mag1".
190716-16:24:51,708 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_3.ds_mag1".
190716-16:24:51,712 nipype.workflow INFO:
	 [Node] Running "ds_mag2" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-16:24:51,717 nipype.workflow INFO:
	 [Node] Running "ds_mag1" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-

/Users/gdehol/miniconda3/lib/python3.7/site-packages/nipype/interfaces/fsl/preprocess.py:1887: UserWarning: This has not been fully tested. Please report any failures.
  warn('This has not been fully tested. Please report any failures.')


190716-16:25:01,757 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_7.ds_mag1".
190716-16:25:01,762 nipype.workflow INFO:
	 [Job 2] Cached (unwrap_16_7.norm_phase2).
190716-16:25:01,759 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_16_7.ds_mag2" in "/Users/gdehol/workflow_folders/unwrap_16_7/ds_mag2".
190716-16:25:01,766 nipype.workflow INFO:
	 [Job 4] Cached (unwrap_16_7.norm_phase1).
190716-16:25:01,765 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_7.ds_mag2".
190716-16:25:01,764 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_16_7.ds_mag1" in "/Users/gdehol/workflow_folders/unwrap_16_7/ds_mag1".
190716-16:25:01,770 nipype.workflow INFO:
	 [Node] Outdated cache found for "unwrap_16_7.ds_mag1".
190716-16:25:01,774 nipype.workflow INFO:
	 [Node] Running "ds_mag2" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-16:25:01,780 nipype.workflow INFO:
	 [Node] Running "ds_mag1" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-

...

190716-17:20:33,723 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_2.ds_mag2" in "/Users/gdehol/workflow_folders/unwrap_28_2/ds_mag2".
190716-17:20:33,726 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_2.ds_mag1" in "/Users/gdehol/workflow_folders/unwrap_28_2/ds_mag1".
190716-17:20:33,728 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_2.norm_phase2" in "/Users/gdehol/workflow_folders/unwrap_28_2/norm_phase2".
190716-17:20:33,729 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_2.norm_phase1" in "/Users/gdehol/workflow_folders/unwrap_28_2/norm_phase1".
190716-17:20:33,736 nipype.workflow INFO:
	 [Node] Running "ds_mag2" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-17:20:33,738 nipype.workflow INFO:
	 [Node] Running "ds_mag1" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-17:20:33,741 nipype.workflow INFO:
	 [Node] Running "norm_phase2" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/risk_pre

/Users/gdehol/miniconda3/lib/python3.7/site-packages/nipype/interfaces/fsl/preprocess.py:1887: UserWarning: This has not been fully tested. Please report any failures.
  warn('This has not been fully tested. Please report any failures.')


190716-17:20:51,815 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_6.ds_mag2" in "/Users/gdehol/workflow_folders/unwrap_28_6/ds_mag2".
190716-17:20:51,818 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_6.ds_mag1" in "/Users/gdehol/workflow_folders/unwrap_28_6/ds_mag1".
190716-17:20:51,820 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_6.norm_phase2" in "/Users/gdehol/workflow_folders/unwrap_28_6/norm_phase2".
190716-17:20:51,821 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_6.norm_phase1" in "/Users/gdehol/workflow_folders/unwrap_28_6/norm_phase1".
190716-17:20:51,826 nipype.workflow INFO:
	 [Node] Running "ds_mag2" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-17:20:51,829 nipype.workflow INFO:
	 [Node] Running "ds_mag1" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-17:20:51,833 nipype.workflow INFO:
	 [Node] Running "norm_phase1" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/risk_pre

/Users/gdehol/miniconda3/lib/python3.7/site-packages/nipype/interfaces/fsl/preprocess.py:1887: UserWarning: This has not been fully tested. Please report any failures.
  warn('This has not been fully tested. Please report any failures.')


190716-17:21:09,916 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_11.ds_mag2" in "/Users/gdehol/workflow_folders/unwrap_28_11/ds_mag2".
190716-17:21:09,920 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_11.ds_mag1" in "/Users/gdehol/workflow_folders/unwrap_28_11/ds_mag1".
190716-17:21:09,920 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_11.norm_phase2" in "/Users/gdehol/workflow_folders/unwrap_28_11/norm_phase2".
190716-17:21:09,921 nipype.workflow INFO:
	 [Node] Setting-up "unwrap_28_11.norm_phase1" in "/Users/gdehol/workflow_folders/unwrap_28_11/norm_phase1".
190716-17:21:09,929 nipype.workflow INFO:
	 [Node] Running "ds_mag2" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-17:21:09,933 nipype.workflow INFO:
	 [Node] Running "ds_mag1" ("niworkflows.interfaces.bids.DerivativesDataSink")
190716-17:21:09,933 nipype.workflow INFO:
	 [Node] Running "norm_phase1" ("nipype.interfaces.fsl.utils.ImageMaths"), a CommandLine Interface with command:
fslmaths /data/